In [18]:
#%% Import libraries
import numpy as np
from sympy import *
# from sympy.physics.mechanics import ReferenceFrame
from IPython.display import display, Latex, Math
init_printing(use_latex='mathjax')

from sys import path
path.append(
    "c:\\Users\\diego\\Dropbox\\Academic\\MEng Space Systems\\3. DOCA\\ADCS functions")
import ADCS_Functions as adcs
import ADCS_Functions_sym as adcs_sym

_________________________________

A spacecraft with a principal axes body-fixed frame b, has corresponding 
principal moments of inertia Ix, Iy, Iz. 

The spacecraft attitude relative to the Earth centered inertial frame G is 
described by a yaw-pitch-roll (3-2-1) Euler sequence, represented by the 
rotation matrix

In [19]:
Ix = 100 # kg*m^2
Iy = 120 # kg*m^2
Iz = 80 # kg*m^2

I = np.array([[Ix, 0, 0], [0, Iy, 0], [0, 0, Iz]])

In [20]:
CbG = adcs_sym.DCM('sim', 3, 2, 1, invorder=True)
CbG

⎡           cos(y)⋅cos(z)                          sin(z)⋅cos(y)              
⎢                                                                             
⎢sin(x)⋅sin(y)⋅cos(z) - sin(z)⋅cos(x)  sin(x)⋅sin(y)⋅sin(z) + cos(x)⋅cos(z)   
⎢                                                                             
⎣sin(x)⋅sin(z) + sin(y)⋅cos(x)⋅cos(z)  -sin(x)⋅cos(z) + sin(y)⋅sin(z)⋅cos(x)  

   -sin(y)   ⎤
             ⎥
sin(x)⋅cos(y)⎥
             ⎥
cos(x)⋅cos(y)⎦

Where sb = sin b and cb = cos b. Where φ, Θ and ψ are the roll, pitch and yaw
angles, respectively. Currently, the attitude is represented by
φ = Θ = ψ = π/4 rad, and the spacecraft orbital position ( in ECI) coordinates is

In [21]:
R_0sim = Symbol('R_0')

R_0G_vecsim = Matrix([0, 0 , R_0sim]) # km
R_0G_vecsim

⎡0 ⎤
⎢  ⎥
⎢0 ⎥
⎢  ⎥
⎣R₀⎦

In [22]:
R_0 = 7000
R_0G_vec = R_0G_vecsim.subs(R_0sim, R_0)  # km
R_0G_vec = adcs.print_matrix(np.array(R_0G_vec).astype(np.float64))
R_0G_vec 


<IPython.core.display.Math object>

Determine the gravity-gradient torque acting on the spacecraft. Express the 
result in spacecraft body coordinates. Note that μ = 3.986 ∙ 105 km3/s2 is the 
Earth ́s gravitational constant

In [23]:
mu = 3.986004418e5 # km^3/s^2
mu

398600.4418

___________________

In body coordinates, the gravity-gradient torque is given by:

$\mathbf{T}_{g g, b}=\frac{3 \mu}{R_{o}^{5}} \mathbf{R}_{o, b}^{\times} \mathbf{I} \mathbf{R}_{o, b}$

Where $I = diag\{I_x, I_y, I_z\}$ is the spacecraft inertia matrix, $R_o$ is the spacecraft 
orbital position vector in body coordinates $R_o=|R_o|$, and $\mu$ is Earth ́s 
gravitational constant. We are given the spacecraft orbital position in ECI 
coordinates, namely: $R_o = \begin{bmatrix}o\\o\\R_0\end{bmatrix}$

In body coordinates, therfore, we have:

In [24]:
R_0b_vec = CbG*R_0G_vecsim
R_0b_vec

⎡   -R₀⋅sin(y)   ⎤
⎢                ⎥
⎢R₀⋅sin(x)⋅cos(y)⎥
⎢                ⎥
⎣R₀⋅cos(x)⋅cos(y)⎦

We are gioven that $x=y=z=\pi/4$

In [25]:
x,y,z = symbols('x y z')

R_0b_vec = R_0b_vec.subs(x, pi/4).subs(y, pi/4).subs(z, pi/4)
R_0b_vec


⎡-√2⋅R₀ ⎤
⎢───────⎥
⎢   2   ⎥
⎢       ⎥
⎢  R₀   ⎥
⎢  ──   ⎥
⎢  2    ⎥
⎢       ⎥
⎢  R₀   ⎥
⎢  ──   ⎥
⎣  2    ⎦

In [26]:
R_0b_vec = adcs_sym.sym2num(R_0b_vec, R_0sim, R_0)
adcs.print_matrix(R_0b_vec)


<IPython.core.display.Math object>

In [27]:
np.linalg.norm(adcs_sym.sym2num(R_0G_vecsim, R_0sim, R_0))


7000.0

In [28]:
adcs.print_matrix(adcs.skew(R_0b_vec))


<IPython.core.display.Math object>

Therefore, the gravity-gradient in body coordinates is:

In [29]:
Tgg_b = (3*mu) / (R_0**5) * adcs.skew(R_0b_vec) @ I @ R_0b_vec
Tgg_b


array([-3.48630124e-05, -2.46518725e-05, -2.46518725e-05])

In [30]:
# Or using the library:

adcs.gravity_gradient_Tb(I, R_0b_vec)

array([-3.48629738e-05, -2.46518452e-05, -2.46518452e-05])

In [33]:
# If using small angle approximation:

adcs.gravity_gradient_Tb_aprox(np.array([Ix, Iy, Iz]), np.array([np.pi/4, np.pi/4, np.pi/4]), R_0)

array([-1.09525262e-04, -5.47626311e-05,  0.00000000e+00])

As we can see, the gravity-gradient torque cannot be approximated with small angles when the angle is pi/4. Only when it is close to 0